In [ ]:
#Import Libraries

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import random
import matplotlib.pyplot as plt
import seaborn as sns
random.seed(1)


# Load the SAS XPT file
data = pd.read_sas("/Users/troytoy/Downloads/LLCP2023.XPT", format="xport")


/opt/anaconda3/lib/python3.12/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [ ]:
# Select specific columns for processing
brfss_df_selected = data[['DIABETE4',        # Diagnosed with diabetes status (Yes/No/Borderline)
                          '_BMI5',          # Body Mass Index (BMI) calculated
                          '_RFBMI5',        # Risk factor for obesity (BMI > 30)
                          '_RFHYPE6',       # Hypertension status (Yes/No)
                          'TOLDHI3',        # High cholesterol diagnosis (Yes/No)
                          '_CHOLCH3',       # Cholesterol checked in past 5 years
                          'CHCKDNY2',       # Chronic kidney disease diagnosis (Yes/No)
                          'SMOKE100',       # Smoked 100 cigarettes in lifetime (Yes/No)
                          '_RFDRHV8',       # Heavy alcohol drinking status
                          'CVDSTRK3',       # Ever had a stroke (Yes/No)
                          '_MICHD',         # Ever had myocardial infarction (Yes/No)
                          '_TOTINDA',       # Total physical activity indicator (Active/Inactive)
                          'GENHLTH',        # General health status (Excellent to Poor)
                          'PHYSHLTH',       # Number of physically unhealthy days in the past 30 days
                          'MENTHLTH',       # Number of mentally unhealthy days in the past 30 days
                          'DIFFWALK',       # Difficulty walking or climbing stairs (Yes/No)
                          '_HLTHPL1',       # Has any health care coverage (Yes/No)
                          'MEDCOST1',       # Couldn't see a doctor due to cost (Yes/No)
                          'CHECKUP1',       # Last routine check-up time frame
                          '_SEX',           # Sex of respondent (Male/Female)
                          '_AGEG5YR',       # Age group (5-year intervals)
                          'EDUCA',          # Education level
                          'INCOME3']]       # Income level

In [ ]:

#check how many rows and columns are left
brfss_df_selected.shape

In [ ]:
brfss_df_selected.head()

In [ ]:
#Drop missing values from the datafame - knocks 100,000 rows right away
brfss_df_selected = brfss_df_selected.dropna()
brfss_df_selected.shape

In [ ]:
# DIABETE4 : Diabetes Awareness

'''Making this ordinal. Set :
0 for no diabetes or for only during pregnancy, 
1 for pre-diabetes or borderline diabetes, 
2 is for yes diabetes. '''
# Remove all 7 (don't knows)
# remove all 9 (refused to answer)

brfss_df_selected['DIABETE4'] = brfss_df_selected['DIABETE4'].replace({2:0, 3:0, 1:2, 4:1})
#remove rows where the value of the column DIABETE4 is equal to 7 and 9.
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE4 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE4 != 9]
# verify the effectiveness of the filtering operations by displaying the remaining unique values
brfss_df_selected.DIABETE4.unique()

In [ ]:
#Check the shape of the dataset now: We have 234951 cleaned rows and 25 columns (1 of which is our dependent variable)
brfss_df_selected.shape

In [ ]:
#Check how the data looks after Modifying Values
brfss_df_selected.head()

In [ ]:
#Check the size of each class for the diabetes column
brfss_df_selected.groupby(['DIABETE4']).size()

In [ ]:
#remane column
brfss = brfss_df_selected.rename(columns={
    'DIABETE4': 'Diabetes_Status',          # Diabetes awareness (No/Pre-diabetes/Diabetes)
    '_BMI5': 'BMI',                         # Body Mass Index
    '_RFBMI5': 'Obesity_Indicator',         # Obesity indicator (BMI >= 30)
    '_RFHYPE6': 'Hypertension',             # Hypertension diagnosis
    'TOLDHI3': 'High_Cholesterol',          # High cholesterol diagnosis
    '_CHOLCH3': 'Cholesterol_Checked',      # Cholesterol checked in the past 5 years
    'CHCKDNY2': 'Chronic_Kidney_Disease',   # Chronic kidney disease diagnosis
    'SMOKE100': 'Smoker_100_Cigarettes',    # Smoked at least 100 cigarettes in lifetime
    '_RFDRHV8': 'Heavy_Alcohol_Consumption',# Heavy alcohol consumption
    'CVDSTRK3': 'Stroke',                   # Stroke diagnosis
    '_MICHD': 'Heart_Disease',              # Coronary heart disease or heart attack
    '_TOTINDA': 'Physical_Activity',        # Physical activity indicator
    'GENHLTH': 'General_Health',            # Self-rated general health status
    'PHYSHLTH': 'Physical_Unhealthy_Days',  # Number of physically unhealthy days
    'MENTHLTH': 'Mental_Unhealthy_Days',    # Number of mentally unhealthy days
    'DIFFWALK': 'Difficulty_Walking',       # Difficulty walking or climbing stairs
    '_HLTHPL1': 'Health_Care_Coverage',     # Health care coverage indicator
    'MEDCOST1': 'Doctor_Visit_Cost_Issue',  # Could not see doctor due to cost
    'CHECKUP1': 'Last_Checkup',             # Last routine check-up timeframe
    '_SEX': 'Gender',                       # Gender (Male/Female)
    '_AGEG5YR': 'Age_Group',                # Age group in 5-year intervals
    'EDUCA': 'Education_Level',             # Education level
    'INCOME3': 'Income_Level'               # Annual income level
})
print("Renamed Columns:")
print(brfss.columns)

In [ ]:
brfss.head()

In [ ]:
brfss.shape

In [ ]:
#Check how many respondents have no diabetes, prediabetes or diabetes.
brfss.groupby(['Diabetes_Status']).size()

In [ ]:
brfss.to_csv('diabetes_BRFSS2023.csv', sep=",", index=False)

In [ ]:
# Copy the old dataframe to a new one
brfss_binary = brfss.copy()

'''Change:
Diabetic (2) to 1, Pre-diabetic (1) to 0
to have 0 for no diabetes and pre-diabetes, and 1 for diabetes.'''
brfss_binary['Diabetes_Status'] = brfss_binary['Diabetes_Status'].replace({1: 0, 2: 1})

# Rename the column to Diabetes_binary
brfss_binary = brfss_binary.rename(columns={'Diabetes_Status': 'Diabetes_binary'})

# Check unique values in the updated Diabetes_binary column
print("Unique values in Diabetes_binary column:")
print(brfss_binary.Diabetes_binary.unique())

In [ ]:
#1 _BMI5 : Body mass index 
# BMI values are BMI * 100 (i.e. BMI of 1850 is really 18.50)
# Divide BMI by 100 and round them to the nearest whole number.

brfss_df_selected['_BMI5'] = brfss_df_selected['_BMI5'].div(100).round(0)
brfss_df_selected._BMI5.unique()

In [ ]:
#2 _RFBMI5 : Body mass index(BMI) greater than 25.00 (Overweight or Obese)
'''Change 1 to 0 for not overweight and obese(1200 <= BMI < 2500)
2 to 1 for yes overweight or obese'''
# Remove all 9 (don't knows and refused to answer)

brfss_df_selected['_RFBMI5'] = brfss_df_selected['_RFBMI5'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFBMI5 != 9]
brfss_df_selected._RFBMI5.unique()

In [ ]:
#3 _RFHYPE6 : High Blood Pressure Awareness
'''Change :
1 to 0 for no high blood pressure, 
2 to 1 for high blood pressure'''
# Remove all 9 (don't knows and refused to answer)

brfss_df_selected['_RFHYPE6'] = brfss_df_selected['_RFHYPE6'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFHYPE6 != 9]
brfss_df_selected._RFHYPE6.unique()

In [ ]:
#4 TOLDHI3 : Cholesterol Awareness
'''It has 1 for high cholesterol
only change 2 to 0 for no high cholesterol'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['TOLDHI3'] = brfss_df_selected['TOLDHI3'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI3 != 9]
brfss_df_selected.TOLDHI3.unique()

In [ ]:
#5 _CHOLCH3 : Cholesterol check within past five years
'''It has 1 for cholesterol checked in past 5 years,
change and 2,3 to 0 for not checked cholesterol in past 5 years'''
# Remove all 7 and 9 (don't knows and refused to answer)

# Replace values in '_CHOLCH3' column according to mappings
brfss_df_selected['_CHOLCH3'] = brfss_df_selected['_CHOLCH3'].replace({2: 0, 3: 0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._CHOLCH3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected._CHOLCH3 != 9]
brfss_df_selected._CHOLCH3.unique()

In [ ]:
#6 CHCKDNY2 : Unable to control urine flow or kidney disease
'''It has 1 for yes uncontrollable urine flow
change 2 to 0 for no kidney disease (normal urine flow)'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['CHCKDNY2'] = brfss_df_selected['CHCKDNY2'].replace({2: 0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.CHCKDNY2 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.CHCKDNY2 != 9]
brfss_df_selected.CHCKDNY2.unique()

In [ ]:
#7 SMOKE100 : Smoked at least 100 Cigarettes
'''It has 1 for yes smoked
change 2 to 0 for not smoked'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['SMOKE100'] = brfss_df_selected['SMOKE100'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 9]
brfss_df_selected.SMOKE100.unique()

In [ ]:
# 8. _RFDRHV8 : Heavy Alcohol Consumption
'''
Making this binary:
0 for not a heavy drinker,
1 for heavy drinker.
'''
# Remove all 9 (don't knows and refused to answer)
brfss_df_selected['_RFDRHV8'] = brfss_df_selected['_RFDRHV8'].replace({2:1, 1:0})

# Remove rows where the value of _RFDRHV8 is equal to 9 (Don't Know/Refused/Missing)
brfss_df_selected = brfss_df_selected[brfss_df_selected['_RFDRHV8'] != 9]

# Verify the effectiveness of the filtering by displaying the remaining unique values
print("Unique values in _RFDRHV8 after transformation:")
print(brfss_df_selected['_RFDRHV8'].unique())

In [ ]:
#9 CVDSTRK3 : Chronic health conditions
'''It has 1 for yes respondant has had a stroke
only change 2 to 0 for no'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['CVDSTRK3'] = brfss_df_selected['CVDSTRK3'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.CVDSTRK3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.CVDSTRK3 != 9]
brfss_df_selected.CVDSTRK3.unique()

In [ ]:
#10 _MICHD : Ever has Coronary Heart Disease (CHD) or Myocardial Infarction(MI)
'''It has 1 for yes respondant has CHD or MI
Change 2 to 0 for respondant does not have CHD or MI'''
brfss_df_selected['_MICHD'] = brfss_df_selected['_MICHD'].replace({2: 0})
brfss_df_selected._MICHD.unique()

In [ ]:
#11 _TOTINDA : Does physical activity or exercise
'''It has 1 for yes respondant does physical activity or excercise
Change 2 to 0 for not involved in any physical activity or excercise'''
# Remove all 9 (don't knows and refused to answer)

brfss_df_selected['_TOTINDA'] = brfss_df_selected['_TOTINDA'].replace({2: 0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._TOTINDA != 9]
brfss_df_selected._TOTINDA.unique()

In [ ]:
#12 GENHLTH : Genreal Health Status
'''This is an ordinal column which will remain as it is
(1 = Excellent, 2 = Very good, 3 = Good, 4 = Fair, 5 = Poor)'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 9]
brfss_df_selected.GENHLTH.unique()

In [ ]:
# 13 PHYSHLTH : Number of days Physical Health is good
'''It has a scale of 0-30 days for good physical health days
change 88 to 0 for no as repondant has bad health days'''
# Remove all 77 and 99 (don't knows and refused to answer)

brfss_df_selected['PHYSHLTH'] = brfss_df_selected['PHYSHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 99]
brfss_df_selected.PHYSHLTH.unique()

In [ ]:
#14 MENTHLTH : Number of days Mental Health is not good
'''It has a scale of 0-30 days for bad mental health days
change 88 to 0 for no as repondant has no bad mental health days(good mental health days)'''
# Remove all 77 and 99 (don't knows and refused to answer)

brfss_df_selected['MENTHLTH'] = brfss_df_selected['MENTHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 99]
brfss_df_selected.MENTHLTH.unique()

In [ ]:
#15 DIFFWALK : Difficulty Walking or climbing stairs
'''It has 1 for yes difficulty walking or climbing stairs
change 2 to 0 for no difficulty walking or climbing stairs'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['DIFFWALK'] = brfss_df_selected['DIFFWALK'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 9]
brfss_df_selected.DIFFWALK.unique()

In [ ]:
#16 _HLTHPL1 : Have any health care access or insurance
'''It has 1 for yes respondant has health care access
change 2 to 0 for no health care access'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['_HLTHPL1'] = brfss_df_selected['_HLTHPL1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._HLTHPL1 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected._HLTHPL1 != 9]
brfss_df_selected._HLTHPL1.unique()

In [ ]:
#17 MEDCOST1 : Couldn't afford to see a doctor
'''It has 1 for yes 
change 2 to 0 for no couldn't afford to see a doctor when needed'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['MEDCOST1'] = brfss_df_selected['MEDCOST1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST1 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST1 != 9]
brfss_df_selected.MEDCOST1.unique()

In [ ]:
#18 CHECKUP1 : Last routine check up
'''It is already an ordinal column with:
1 = Within past year (anytime less than 12 months ago)
2 = Within past 2 years (1 year but less than 2 years ago)
3 = Within past 5 years (2 years but less than 5 years ago)
4 = 5 or more years ago
Change 8 to 5 for Never'''
# Remove all 7 and 9 (don't knows and refused to answer)

brfss_df_selected['CHECKUP1'] = brfss_df_selected['CHECKUP1'].replace({8:5})
brfss_df_selected = brfss_df_selected[brfss_df_selected.CHECKUP1 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.CHECKUP1 != 9]
brfss_df_selected.CHECKUP1.unique()

In [ ]:
#19 _SEX : Respondant's SEX
'''Set 0 for female (change 2 to 0)
and 1 for male because men comparatively are at higher risk for heart disease'''
brfss_df_selected['_SEX'] = brfss_df_selected['_SEX'].replace({2:0})
brfss_df_selected._SEX.unique()

In [ ]:
#20 _AGEG5YR : Reported age in five-year age categories
'''It is already ordinal with 5 year interval
It starts with 1 for age 18-24 all the way to 13 for 80 years and older'''
# Remove 14 for missing and don't know values

brfss_df_selected = brfss_df_selected[brfss_df_selected._AGEG5YR != 14]
brfss_df_selected._AGEG5YR.unique()

In [ ]:
#21 EDUCA : Education Level
'''It is an ordinal column with: 
1 = never attended education(school or kindergarten)
2 = Elementary school
3 = Some high school
4 = Grade 12 or GED (High school graduate)
5 = College 1 year to 3 years (Some college or technical school)
6 = college graduate(4 years or more)'''
# Remove all 9 (refused to answer)

brfss_df_selected = brfss_df_selected[brfss_df_selected.EDUCA != 9]
brfss_df_selected.EDUCA.unique()

In [ ]:
#22 INCOME3 : Annual Income level
'''It is already ordinal with 1 for less than $10,000 all the way up to 8 being $75,000 or more'''
# Remove all 77 and 99 (don't knows and refused to answer)

brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME3 != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME3 != 99]
brfss_df_selected.INCOME3.unique()

In [ ]:
#Check the shape of the dataset now: We have 234951 cleaned rows and 25 columns (1 of which is our dependent variable)
brfss_df_selected.shape

In [ ]:
#Check how the data looks after Modifying Values
brfss_df_selected.head()

In [ ]:
#Check the size of each class for the diabetes column
brfss_df_selected.groupby(['DIABETE4']).size()

In [ ]:
brfss = brfss_df_selected.rename(columns={
    'DIABETE4': 'Diabetes_Status',          # Diabetes awareness (No/Pre-diabetes/Diabetes)
    '_BMI5': 'BMI',                         # Body Mass Index
    '_RFBMI5': 'Obesity_Indicator',         # Obesity indicator (BMI >= 30)
    '_RFHYPE6': 'Hypertension',             # Hypertension diagnosis
    'TOLDHI3': 'High_Cholesterol',          # High cholesterol diagnosis
    '_CHOLCH3': 'Cholesterol_Checked',      # Cholesterol checked in the past 5 years
    'CHCKDNY2': 'Chronic_Kidney_Disease',   # Chronic kidney disease diagnosis
    'SMOKE100': 'Smoker_100_Cigarettes',    # Smoked at least 100 cigarettes in lifetime
    '_RFDRHV8': 'Heavy_Alcohol_Consumption',# Heavy alcohol consumption
    'CVDSTRK3': 'Stroke',                   # Stroke diagnosis
    '_MICHD': 'Heart_Disease',              # Coronary heart disease or heart attack
    '_TOTINDA': 'Physical_Activity',        # Physical activity indicator
    'GENHLTH': 'General_Health',            # Self-rated general health status
    'PHYSHLTH': 'Physical_Unhealthy_Days',  # Number of physically unhealthy days
    'MENTHLTH': 'Mental_Unhealthy_Days',    # Number of mentally unhealthy days
    'DIFFWALK': 'Difficulty_Walking',       # Difficulty walking or climbing stairs
    '_HLTHPL1': 'Health_Care_Coverage',     # Health care coverage indicator
    'MEDCOST1': 'Doctor_Visit_Cost_Issue',  # Could not see doctor due to cost
    'CHECKUP1': 'Last_Checkup',             # Last routine check-up timeframe
    '_SEX': 'Gender',                       # Gender (Male/Female)
    '_AGEG5YR': 'Age_Group',                # Age group in 5-year intervals
    'EDUCA': 'Education_Level',             # Education level
    'INCOME3': 'Income_Level'               # Annual income level
})
print("Renamed Columns:")
print(brfss.columns)

In [ ]:
brfss.head()

In [ ]:
brfss.shape

In [ ]:
#Check how many respondents have no diabetes, prediabetes or diabetes.
brfss.groupby(['Diabetes_Status']).size()

In [ ]:
brfss.to_csv('diabetes_BRFSS2023.csv', sep=",", index=False)

In [ ]:
#Copy the old dataframe to a new one.
brfss_binary = brfss
'''Change:
diabetic 2 to 1, pre-diabetic 1 to 0
in order to have 0 for no diabetes and pre-diabetes and 1 for respondant has diabetes'''

brfss_binary['Diabetes_Status'] = brfss_binary['Diabetes_Status'].replace({1:0, 2:1})

#Rename the column to DiabetesBinary
brfss_binary = brfss_binary.rename(columns = {'Diabetes_Status': 'Diabetes_binary'})
brfss_binary.Diabetes_binary.unique()

In [ ]:
brfss_binary.head()

In [ ]:
#Check size of class
brfss_binary.groupby(['Diabetes_binary']).size()

In [ ]:
#Separate the Not Diabetes: 0s ;  Pre-diabetes and Diabetes: 1s, 2s
#Get the 1s
diabetes_mask = brfss_binary['Diabetes_binary'] == 1
brfss_diabetes = brfss_binary[diabetes_mask]

#Get the 0s
no_diabetes_mask = brfss_binary['Diabetes_binary'] == 0
brfss_no_diabetes = brfss_binary[no_diabetes_mask]

'''Select 37909 random cases from the non-diabetes group i.e. 0
we already have 37909 cases from the diabetes risk group'''
brfss_no_diabetes_sampled = brfss_no_diabetes.take(np.random.permutation(len(brfss_no_diabetes))[:37909])

# Append the 37909 1s(diabetes) to the 37909 randomly selected 0s(no diabetes)
brfss_balanced = brfss_no_diabetes_sampled.append(brfss_diabetes, ignore_index=True)

In [ ]:
#Now we have a dataset of 75818 rows that is equally balanced with 50% 1s and 50% 0s for the target variable Diabetes_binary
brfss_balanced

In [ ]:
#See the perfectly balanced class
brfss_balanced.groupby(['Diabetes_binary']).size()

In [ ]:
print(f'brfss_5050={brfss_balanced.shape}',f'brfss_binary={brfss_binary.shape}')

In [ ]:
#Binary imbalanced dataset (cleaned)
# 2nd save version
brfss_binary.to_csv('diabetes_binary_BRFSS2023.csv', sep=",", index=False)

In [ ]:
#Binary balanced dataset (cleaned)
# 3rd save version
brfss_balanced.to_csv('diabetes_binary_5050split_BRFSS2023.csv', sep=",", index=False)

In [ ]:
# Load the datasets
df_binary = pd.read_csv('diabetes_binary_BRFSS2023.csv')
df_5050 = pd.read_csv('diabetes_binary_5050split_BRFSS2023.csv')

# Display the first few rows of each dataset
df_binary.head()#, df_5050.head()

In [ ]:
df_binary.info(), df_binary.describe()

In [ ]:
#df_5050.info(), df_5050.describe()

In [ ]:
# Correlation heatmap for df_binary
numeric_df_binary = df_binary.select_dtypes(include=[np.number])
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_df_binary.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap for df_binary')
plt.show()

In [ ]:
target_correlations = numeric_df_binary.corr()['Diabetes_binary'].sort_values(ascending=False)
print("Correlations with diabetes_binary:")
print(target_correlations)

In [ ]:
# Exclude the target variable itself and select the top 5
top_5_features = target_correlations.index[1:6]  # Skip the first index (target variable itself)
top_5_values = target_correlations[1:6]

# Display the results
print("Top 5 Features Correlated with Diabetes_binary:")
for feature, value in zip(top_5_features, top_5_values):
    print(f"{feature}: {value:.2f}")

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Prepare the DataFrame with the top 5 features
X_top5 = df_binary[top_5_features]

# Add a constant for the intercept
X_top5 = sm.add_constant(X_top5)

# Calculate VIF for each feature
vif_data = pd.DataFrame({
    "Feature": X_top5.columns,
    "VIF": [variance_inflation_factor(X_top5.values, i) for i in range(X_top5.shape[1])]
})

# Display VIF
print("Variance Inflation Factor (VIF) for Top 5 Features:")
print(vif_data)

In [ ]:
X = df_binary[top_5_features]
y = df_binary['Diabetes_binary']  # Dependent variable

# Add a constant for the intercept
X_with_constant = sm.add_constant(X)

# Logistic Regression Model
logit_model = sm.Logit(y, X_with_constant)
result = logit_model.fit()

# Print the summary
print(result.summary())

In [ ]:
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency, ttest_ind

diabetes_group = df_binary[df_binary['Diabetes_binary'] == 1]['General_Health']
no_diabetes_group = df_binary[df_binary['Diabetes_binary'] == 0]['General_Health']

# Perform T-Test
t_stat, p_val = ttest_ind(diabetes_group, no_diabetes_group)

# Print results
print("T-statistic:", t_stat)
print("p-value:", p_val)

# Interpret Results
if p_val < 0.05:
    print("The mean General_Health differs significantly between Diabetes_binary groups.")
else:
    print("No significant difference in General_Health between Diabetes_binary groups.")

In [ ]:
contingency_table = pd.crosstab(df_binary['Diabetes_binary'], df_binary['Hypertension'])

# Perform Chi-Square Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print results
print("Chi-Square Statistic:", chi2)
print("p-value:", p)

# Interpret Results
if p < 0.05:
    print("There is a significant association between Diabetes_Status and Hypertension.")
else:
    print("No significant association between Diabetes_Status and Hypertension.")

In [ ]:
contingency_table = pd.crosstab(df_binary['Diabetes_binary'], df_binary['Age_Group'])

# Perform Chi-Square Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print results
print("Chi-Square Statistic:", chi2)
print("p-value:", p)

# Interpret Results
if p < 0.05:
    print("There is a significant association between Diabetes_Status and Age_Group.")
else:
    print("No significant association between Diabetes_Status and Age_Group.")

In [ ]:
contingency_table = pd.crosstab(df_binary['Diabetes_binary'], df_binary['High_Cholesterol'])

# Perform Chi-Square Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print results
print("Chi-Square Statistic:", chi2)
print("p-value:", p)

# Interpret Results
if p < 0.05:
    print("There is a significant association between Diabetes_Status and High_Cholesterol.")
else:
    print("No significant association between Diabetes_Status and High_Cholesterol.")

In [ ]:
# Example: Test mean BMI difference between Diabetes_Status groups
diabetes_group = df_binary[df_binary['Diabetes_binary'] == 1]['BMI']
no_diabetes_group = df_binary[df_binary['Diabetes_binary'] == 0]['BMI']

# Perform T-Test
t_stat, p_val = ttest_ind(diabetes_group, no_diabetes_group)

# Print results
print("T-statistic:", t_stat)
print("p-value:", p_val)

# Interpret Results
if p_val < 0.05:
    print("The mean BMI differs significantly between Diabetes_binary groups.")
else:
    print("No significant difference in BMI between Diabetes_binary groups.")

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Ensure the dataset and variables are correctly defined
# Assuming `df_5050` contains the balanced dataset
# `top_5_features` are the features selected based on correlation analysis

# Initialize the logistic regression model
model = LogisticRegression(max_iter=1000)  # Ensure convergence for complex datasets

# Perform RFE to select the best 5 features
rfe = RFE(estimator=model, n_features_to_select=5)
rfe.fit(df_5050[top_5_features], df_5050['Diabetes_binary'])

# Display selected features
selected_features_rfe = df_5050[top_5_features].columns[rfe.support_]
print("Features Selected by RFE:", selected_features_rfe)

In [ ]:
# Use the final set of features (selected by RFE or based on VIF analysis)
final_features = selected_features_rfe # Alternatively, use manually selected features

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X = df_5050[final_features]
y = df_5050['Diabetes_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model_final = LogisticRegression()
model_final.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
y_pred = model_final.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
#! git init

In [ ]:
! git add DA01_Project.ipynb

In [ ]:
! git add .gitignore

In [ ]:
! git commit -m 'change datasets'

In [ ]:
! git push -u origin main